In [1]:
import numpy as np 

In [2]:
probas = [0.2, 0.8]

np.sum(probas**2)

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [ ]:
class Node:
    def __init__(self):
        
        # links to the left and right child nodes
        self.right = None
        self.left = None
        
        # derived from splitting criteria
        self.column = None
        self.threshold = None
        
        # probability for object inside the Node to belong for each of the given classes
        self.probas = None
        # depth of the given node
        self.depth = None
        
        # if it is the root Node or not
        self.is_terminal = False

In [ ]:
class DecisionTreeClassifier:
    def __init__(self, max_depth = 3, min_samples_leaf = 1, min_samples_split = 2):
        
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        
        # Decision tree itself
        self.Tree = None
    
    '''
    y = [1,0,1,0,1] 이라면... 
    P(0|y) = num(zeros) / len(y) = 2/5 
    P(1|y) = num(oness) / len(y) = 3/5
    => probas = [2/5, 3/5] 이다. 
    '''
    def nodeProbas(self, y):
        probas = [] 
        for one_class in self.classes:
            proba = y[y==one_class].shape[0] / y.shape[0]
            probas.append(proba)
        return probas
    
    def gini(self, probas):
        tot_gini =0
        for proba_class in probas:
            tot_gini+=1 - proba_class**2
        
        pass
    
    def calcImpurity(self, y):
        '''
        Wrapper for the impurity calculation. Calculates probas first and then passses them
        to the Gini criterion
        '''
        pass
    
    def calcBestSplit(self, X, y):
        '''
        Calculates the best possible split for the concrete node of the tree
        '''
        
        pass
    
    def buildDT(self, X, y, node):
        '''
        Recursively builds decision tree from the top to bottom
        '''
        
        pass
    
    def fit(self, X, y):
        '''
        Standard fit function to run all the model training
        '''
        
        pass
    
    def predictSample(self, x, node):
        '''
        Passes one object through decision tree and return the probability of it to belong to each class
        '''
       
        pass
    
    def predict(self, X):
        '''
        Returns the labels for each X
        '''
        
        pass

In [3]:
class DecisionTreeClassifier:
    def __init__(self, max_depth = 3, min_samples_leaf = 1, min_samples_split = 2):
        
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        
        self.classes = None
        
        # Decision tree itself
        self.Tree = None
    
    def nodeProbas(self, y):
        '''
        Calculates probability of class in a given node
        '''
        
        probas = []
        
        # for each unique label calculate the probability for it
        for one_class in self.classes:
            proba = y[y == one_class].shape[0] / y.shape[0]
            probas.append(proba)
        return np.asarray(probas)

    def gini(self, probas):
        '''
        Calculates gini criterion
        '''
        
        return 1 - np.sum(probas**2)
    
    def calcImpurity(self, y):
        '''
        현재 이 y 는 target만 존재하는 vector (1차원짜리 pd.dataFrame) 로써, 
        1,0 이 모두 섞여 있다.
        '''
        
        return self.gini(self.nodeProbas(y))
    
    def calcBestSplit(self, X, y):
        '''
        Calculates the best possible split for the concrete node of the tree
        '''
        
        bestSplitCol = None
        bestThresh = None
        bestInfoGain = -999
        
        # 나누기 전 현재의 데이터 분포에서 impurity 를 계산한다. 
        impurityBefore = self.calcImpurity(y)
        
        # X는 2차원의 table 데이터 이므로, (행 수, 열 수) 의 shape을 갖는다. 
        # 따라서, 모든 variable 들에 대해서 info gain을 계산해야 하므로 loop 돌린다. 
        for col in range(X.shape[1]):
            x_col = X[:, col]  # 행은 전부다, 열은 col으로 지정한 열만. 
            
            # 각각의 모든 value 들을 기준로 나눈다. 
            for x_i in x_col:
                threshold = x_i
                y_right = y[x_col > threshold]
                y_left = y[x_col <= threshold]
                
                if y_right.shape[0] == 0 or y_left.shape[0] == 0:
                    continue
                    
                # calculate impurity for the right and left nodes
                impurityRight = self.calcImpurity(y_right)
                impurityLeft = self.calcImpurity(y_left)
                
                # calculate information gain
                infoGain = impurityBefore
                infoGain -= (impurityLeft * y_left.shape[0] / y.shape[0]) + (impurityRight * y_right.shape[0] / y.shape[0])
                
                # is this infoGain better then all other?
                if infoGain > bestInfoGain:
                    bestSplitCol = col
                    bestThresh = threshold
                    bestInfoGain = infoGain
                    
        
        # if we still didn't find the split
        if bestInfoGain == -999:
            return None, None, None, None, None, None
        
        # making the best split
        
        x_col = X[:, bestSplitCol]
        x_left, x_right = X[x_col <= bestThresh, :], X[x_col > bestThresh, :]
        y_left, y_right = y[x_col <= bestThresh], y[x_col > bestThresh]
        
        return bestSplitCol, bestThresh, x_left, y_left, x_right, y_right
                
                
                
    
    def buildDT(self, X, y, node):
        '''
        Recursively builds decision tree from the top to bottom
        '''
        
        # checking for the terminal conditions
        
        if node.depth >= self.max_depth:
            node.is_terminal = True
            return
        
        if X.shape[0] < self.min_samples_split:
            node.is_terminal = True
            return
        
        if np.unique(y).shape[0] == 1:
            node.is_terminal = True
            return
        
        # calculating current split
        splitCol, thresh, x_left, y_left, x_right, y_right = self.calcBestSplit(X, y)
        
        if splitCol is None:
            node.is_terminal = True
            
        if x_left.shape[0] < self.min_samples_leaf or x_right.shape[0] < self.min_samples_leaf:
            node.is_terminal = True
            return
        
        node.column = splitCol
        node.threshold = thresh
        
        # creating left and right child nodes
        node.left = Node()
        node.left.depth = node.depth + 1
        node.left.probas = self.nodeProbas(y_left)
        
        node.right = Node()
        node.right.depth = node.depth + 1
        node.right.probas = self.nodeProbas(y_right)
        
        # splitting recursevely
        self.buildDT(x_right, y_right, node.right)
        self.buildDT(x_left, y_left, node.left)
        
        
        
        
    
    def fit(self, X, y):
        '''
        X : y만 빠진 column들이 모두 존재하는 pandas dataframe.
        y : y 하나만 존재하는 pandas dataframe
        '''
        
        if type(X) == pd.DataFrame:
            X = np.asarray(X)
        
        self.classes = np.unique(y)
        # root node creation
        self.Tree = Node()
        self.Tree.depth = 1
        
        # 현재 root node, 즉 모든 데이터에 대해서 probas 를 계산한다. 
        self.Tree.probas = self.nodeProbas(y)
        
        # 그렇게 root node 를 손수 만들어준 다음, 그 root node 를 기준으로 tree를 만든다. 
        self.buildDT(X, y, self.Tree)
    
    def predictSample(self, x, node):
        '''
        Passes one object through decision tree and return the probability of it to belong to each class
        '''
       
    
        # if we have reached the terminal node of the tree
        if node.is_terminal:
            return node.probas
        
        if x[node.column] > node.threshold:
            probas = self.predictSample(x, node.right)
        else:
            probas = self.predictSample(x, node.left)
            
        return probas
        
        
    
    def predict(self, X):
        '''
        Returns the labels for each X
        '''
        
        if type(X) == pd.DataFrame:
            X = np.asarray(X)
            
        predictions = []
        for x in X:
            pred = np.argmax(self.predictSample(x, self.Tree))
            predictions.append(pred)
        
        return np.asarray(predictions)

In [8]:
class Node:
    def __init__(self):
        
        # links to the left and right child nodes
        self.right = None
        self.left = None
        
        # derived from splitting criteria
        self.column = None
        self.threshold = None
        
        # probability for object inside the Node to belong for each of the given classes
        self.probas = None
        # depth of the given node
        self.depth = None
        
        # if it is the root Node or not
        self.is_terminal = False

In [5]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

data = load_iris()
X, y, column_names = data['data'], data['target'], data['feature_names']
X = pd.DataFrame(X, columns = column_names)
X['target'] = y

In [17]:
data["data"].shape

(150, 4)

In [6]:
X, y = X.drop(columns = 'target'), X['target']

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y, random_state = 44)

In [9]:

%%time
model = DecisionTreeClassifier(max_depth = 10, min_samples_leaf=2, min_samples_split=2)
model.fit(X_train, y_train)

CPU times: user 1.96 s, sys: 0 ns, total: 1.96 s
Wall time: 1.96 s


In [10]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_val)
print(f'Accuracy for self built model {accuracy_score(y_val, y_pred)}')

Accuracy for self built model 0.9473684210526315


In [11]:
X_train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
123,6.3,2.7,4.9,1.8
80,5.5,2.4,3.8,1.1
43,5.0,3.5,1.6,0.6
6,4.6,3.4,1.4,0.3
112,6.8,3.0,5.5,2.1
...,...,...,...,...
84,5.4,3.0,4.5,1.5
96,5.7,2.9,4.2,1.3
131,7.9,3.8,6.4,2.0
59,5.2,2.7,3.9,1.4


In [13]:
y_train

123    2
80     1
43     0
6      0
112    2
      ..
84     1
96     1
131    2
59     1
20     0
Name: target, Length: 112, dtype: int64